In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [ ]:
# Device Setting
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps')
device

In [5]:
# Seed setting
import random

def def_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def_seed(seed=123)

In [8]:
# Hyperparameters for Neural Network
epochs = 100
lr = 0.01
batch_size = 4096

In [12]:
# Data Load
df_train = pd.read_pickle('train.pkl')
df_train = df_train[df_train.columns.drop(['P5'])]

df_test = pd.read_pickle('test.pkl')

## Dataset Setting

In [16]:
class MyDataset(Dataset):
    def __init__(self, df, if_test):
        self.df = df
        self.if_test = if_test
        if self.if_test:
            self.labels = self.df['winning'].values
            self.df = self.df.drop(columns=['winning']).values
        else:
            self.df = self.df.values

    def __getitem__(self, index):
        if self.if_test:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
    
    def __len__(self):
        return len(self.df)